In [21]:
import json
import pandas as pd
from datetime import datetime
from imessages import get_messages

In [ ]:
sender_id = "+15107503277"
query_options = f"dt >= '2025-09-14' AND dt < '2025-09-15' AND sender_id = '{sender_id}'"

payload = get_messages(options=query_options)
messages = payload['messages']
query = payload['query']

df = pd.DataFrame(messages)
df

77


,dt,message_id,sender_id,chat_id,chat_name,is_from_me,content,content_type
0,2025-09-14 00:02:28,30962,+15107503277,None,None,1,bruh i have to wake up early tmr to go hiking,rich_text
1,2025-09-14 00:02:36,30963,+15107503277,None,None,1,i hate hiking,rich_text
2,2025-09-14 00:03:15,30964,+15107503277,None,None,0,dude,rich_text
3,2025-09-14 00:03:18,30965,+15107503277,None,None,0,do u have car,rich_text
4,2025-09-14 00:03:48,30966,+15107503277,None,None,1,most likely yes,rich_text
...,...,...,...,...,...,...,...,...
72,2025-09-14 18:46:40,31048,+15107503277,None,None,1,idk man,plain_text
73,2025-09-14 18:46:56,31049,+15107503277,None,None,1,whatcha doing in saratoga,rich_text
74,2025-09-14 19:12:13,31053,+15107503277,None,None,0,dance practice,plain_text
75,2025-09-14 20:20:11,31057,+15107503277,None,None,1,ahh that’s right,rich_text


In [28]:
def get_most_recent_message(messages, start_index, is_from_me=0):
    while not messages[start_index]["is_from_me"] == is_from_me:
        start_index -= 1
    return messages[start_index]

def calculate_time_diff(msg1, msg2):
    dt1 = datetime.strptime(msg1['dt'], '%Y-%m-%d %H:%M:%S')
    dt2 = datetime.strptime(msg2['dt'], '%Y-%m-%d %H:%M:%S')
    return (dt2 - dt1).total_seconds()

def prepare_for_fine_tuning(messages):
    system_prompt = "You are Sohan's texting bot. Your job is to reply exactly as Sohan would text his girlfriend, Ishani. If Ishani sends multiple texts in a row, you may also reply with multiple texts if needed. Never break character and always respond like Sohan texting his girlfriend."
    system_message = {"role": "system", "content": system_prompt}
    while messages[0]["is_from_me"]:
        del messages[0]

    fine_tuning_data = []
    i = 0

    while i < len(messages):
        if messages[i]["is_from_me"]:
            # prev_message = get_most_recent_message(messages, i, 0)
            # time_diff = calculate_time_diff(prev_message, messages[i])
            fine_tuning_data[-1]["messages"].append({"role": "assistant", "content": messages[i]["content"]})
        elif i > 0 and not messages[i-1]["is_from_me"]:
            fine_tuning_data[-1]["messages"].append({"role": "user", "content": messages[i]["content"]})
        else:
            fine_tuning_data.append({"messages": [
                system_message,
                {"role": "user", "content": messages[i]["content"]}
            ]})
        i += 1

    return fine_tuning_data

In [9]:
fine_tuning_data = prepare_for_fine_tuning(messages)
fine_tuning_data[0]

{'messages': [{'role': 'system',
   'content': "You are Sohan's texting bot. Your job is to reply exactly as Sohan would text his girlfriend, Ishani. If Ishani sends multiple texts in a row, you may also reply with multiple texts if needed. Never break character and always respond like Sohan texting his girlfriend."},
  {'role': 'user', 'content': 'dude'},
  {'role': 'user', 'content': 'do u have car'},
  {'role': 'assistant', 'content': 'most likely yes'}]}

In [30]:
with open('fine_tuning_data.jsonl', 'w') as f:
    for item in fine_tuning_data:
        f.write(json.dumps(item) + '\n')